In [ ]:
# @title Setup

competition = "audio understanding"  # @param
# @markdown ---

from google.colab import userdata
import json

# Get the Kaggle credentials from Colab's userdata
username = userdata.get("KAGGLE_USER")
key = userdata.get("KAGGLE_KEY")

# Echo the credentials into the kaggle.json file
!mkdir -p ~/.kaggle
!echo '{{"username":"{username}","key":"{key}"}}' > ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

competition_name = "audio-understanding"
dir_name = "audio_understand"

!kaggle competitions download -c {competition_name}
!mkdir {dir_name}
!unzip /content/{competition_name}.zip -d {dir_name}
!rm -rf audio-understanding.zip

 86% 340M/395M [00:00<00:00, 1.20GB/s]
100% 395M/395M [00:00<00:00, 1.21GB/s]
Archive:  /content/audio-understanding.zip
  inflating: audio_understand/speechs/speechs/test/015a21ca-7e23-4b70-93e6-da8a1bb8eaab.wav  
  inflating: audio_understand/speechs/speechs/test/02164b42-47ac-4d45-aa3a-0288ede4380e.wav  
  inflating: audio_understand/speechs/speechs/test/0225a06d-7949-49a6-84a0-0f4c2d9005aa.wav  
  inflating: audio_understand/speechs/speechs/test/02270177-6507-4730-b034-404aea8f8a5c.wav  
  inflating: audio_understand/speechs/speechs/test/02c4081a-4df6-45bc-a24d-58efa0d70c1b.wav  
  inflating: audio_understand/speechs/speechs/test/03d93186-3c38-4e6e-925f-152d489ae8ef.wav  
  inflating: audio_understand/speechs/speechs/test/05d3d878-5ecf-4dd8-9663-ccfa7afaa9da.wav  
  inflating: audio_understand/speechs/speechs/test/0620b87a-52ed-46ea-b0bb-8d722b46e55a.wav  
  inflating: audio_understand/speechs/speechs/test/085f8dec-23c1-433c-a3a6-007fe451aef0.wav  
  inflating: audio_understand/spe

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3
    !pip install --no-deps unsloth

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## Overview

In [ ]:
df = pd.read_csv('train_merged_transcription.csv')
print(df.shape)
df.head()

(100, 10)


,id,first_name,last_name,กล่าวสวัสดี,แนะนำชื่อและนามสกุล,บอกประเภทใบอนุญาตและเลขที่ใบอนุญาตที่ยังไม่หมดอายุ,บอกวัตถุประสงค์ของการเข้าพบครั้งนี้,เน้นประโยชน์ว่าลูกค้าได้ประโยชน์อะไรจากการเข้าพบครั้งนี้,บอกระยะเวลาที่ใช้ในการเข้าพบ,transcript
0,d76f8990-f74f-4f11-8bc1-ada4928725ad,วิภาดา,มงคลเกษตริน,False,False,False,True,True,True,วิพดา หรือเรียกว่าวิว ที่ปรึกษาจากธนาคารไทยพาณ...
1,614f96ee-2ea3-4d6a-824d-7b19c59526d8,พิมพ์อร,ชาละนันท์,False,False,False,False,False,True,ฉันชื่อพิมออนนะคะ เรียกออนได้เลย วันนี้ขอเวลาป...
2,70cb44f4-3ec4-4b79-919f-543e67fe7959,สุดารัตน์,เกตุเลฃา,False,False,True,False,False,True,ฉันสุดารัฐชื่อเล่นดาวค่ะ เป็นที่ปรึกษาทางการเง...
3,a478f079-1311-4e32-9038-12afbe70375c,จิตตราพร,ชมะนันทน์,True,False,False,False,False,False,สวัสดีค่ะ คุณสิริพัฒน์ ดิฉันชื่อจิตราพรค่ะ เป็...
4,e5fbd20a-58f4-4bd7-a0c3-9c0985789a67,นคนันทินี,ชพานนท์,True,False,False,False,True,True,สวัสดีค่ะ คุณสิริภา นีน่าจัดธนาคารไทยพาณิชนะคะ...


In [ ]:
prompt = '''### Instruction
你是一名擅长评估银行客户经理（Relationship Manager，简称 RM）与客户之间对话的人工智能助手。

你的任务：,
阅读 RM 与客户的完整对话内容，并根据以下六项标准，判断 RM 是否按照规范完成自我介绍。每项符合标准记为 1 分，未符合标准记为 0 分。

---

评估标准（共 6 项）：,
问候客户
必须以礼貌方式向客户打招呼。,
示例：”您好“、”早上好“、”哈喽，您好“。,
,

介绍自己的姓名（包括姓和名）
只需要评估 RM 是否完整介绍自己，不需要考虑客户的姓名。,
必须同时提供真实的名字和姓氏，昵称或仅名字不视为通过。,
如果名字与姓氏连在一起且没有空格（如“张三丰”），则必须明确标注为姓与名的组合，否则不通过。,
合格示例：“我叫张 伟”，“我是李 小美”。,
不合格示例：“我叫小张”，“你可以叫我小王”，“我叫阿明”。,
,

说明所持许可证类型及有效的许可证编号
必须明确提供许可证编号，并声明该许可证仍在有效期内。,
示例：“我持有编号为 12345 的金融执业许可证，目前仍在有效期内。”,
,

说明此次会面的目的
必须清楚说明此次会面目的，例如：投资组合更新、市场分析、新产品推荐等。,
不允许使用模糊或空泛的表述，如“想来聊聊”、“简单交流一下”。,
合格示例：“今天来是为了更新您的投资组合”、“我们来讨论一下当前市场走势”、“我想介绍一款新产品给您”。,
不合格示例：“今天是来按预约见面”、“我有些事想跟您聊聊”或完全未提及会面目的。,
,

说明客户从此次会面中可获得的好处
必须具体说明客户能从本次会面中获得什么收益。,
示例：“这将有助于您优化投资结构”、“可以帮助提高投资回报率”。,
,

说明此次会面预计所需时间
必须明确表示所需时间（可以是数字或文字），例如：15分钟、半小时、5分钟等。,
若仅说“很快”、“不久”等模糊表述，则不算通过。,
合格示例：“大约需要30分钟”、“预计聊1小时，您方便吗？”,
,

---

评估方法：,
仔细阅读整段对话；,
针对每一条评估标准逐项判断；,
每项给出 1 或 0 分数，代表是否符合要求。,

### Input:
{}
'''

In [ ]:
def prepare_data(df, prompt):
    dataset = []
    for i, row in df.iterrows():
        answer = {
            "is_greeting": row['กล่าวสวัสดี'],
            "is_introself": row['แนะนำชื่อและนามสกุล'],
            "is_informlicense": row['บอกประเภทใบอนุญาตและเลขที่ใบอนุญาตที่ยังไม่หมดอายุ'],
            "is_informobjective": row['บอกวัตถุประสงค์ของการเข้าพบครั้งนี้'],
            "is_informbenefit": row['เน้นประโยชน์ว่าลูกค้าได้ประโยชน์อะไรจากการเข้าพบครั้งนี้'],
            "is_informinterval": row['บอกระยะเวลาที่ใช้ในการเข้าพบ']
        }
        message = {
            'conversations': [
                {"role" : "user", "content" : prompt.format(row['transcript'])},
                {"role" : "assistant", "content" : str(answer)},
            ]
            }
        dataset.append(message)

    return dataset


train_set = prepare_data(df, prompt)
train_set[0]['conversations']

[{'role': 'user',
  'content': '### Instruction\n你是一名擅长评估银行客户经理（Relationship Manager，简称 RM）与客户之间对话的人工智能助手。\n\n你的任务：,\n阅读 RM 与客户的完整对话内容，并根据以下六项标准，判断 RM 是否按照规范完成自我介绍。每项符合标准记为 1 分，未符合标准记为 0 分。\n\n---\n\n评估标准（共 6 项）：,\n问候客户\n必须以礼貌方式向客户打招呼。,\n示例：”您好“、”早上好“、”哈喽，您好“。,\n,\n\n介绍自己的姓名（包括姓和名）\n只需要评估 RM 是否完整介绍自己，不需要考虑客户的姓名。,\n必须同时提供真实的名字和姓氏，昵称或仅名字不视为通过。,\n如果名字与姓氏连在一起且没有空格（如“张三丰”），则必须明确标注为姓与名的组合，否则不通过。,\n合格示例：“我叫张 伟”，“我是李 小美”。,\n不合格示例：“我叫小张”，“你可以叫我小王”，“我叫阿明”。,\n,\n\n说明所持许可证类型及有效的许可证编号\n必须明确提供许可证编号，并声明该许可证仍在有效期内。,\n示例：“我持有编号为 12345 的金融执业许可证，目前仍在有效期内。”,\n,\n\n说明此次会面的目的\n必须清楚说明此次会面目的，例如：投资组合更新、市场分析、新产品推荐等。,\n不允许使用模糊或空泛的表述，如“想来聊聊”、“简单交流一下”。,\n合格示例：“今天来是为了更新您的投资组合”、“我们来讨论一下当前市场走势”、“我想介绍一款新产品给您”。,\n不合格示例：“今天是来按预约见面”、“我有些事想跟您聊聊”或完全未提及会面目的。,\n,\n\n说明客户从此次会面中可获得的好处\n必须具体说明客户能从本次会面中获得什么收益。,\n示例：“这将有助于您优化投资结构”、“可以帮助提高投资回报率”。,\n,\n\n说明此次会面预计所需时间\n必须明确表示所需时间（可以是数字或文字），例如：15分钟、半小时、5分钟等。,\n若仅说“很快”、“不久”等模糊表述，则不算通过。,\n合格示例：“大约需要30分钟”、“预计聊1小时，您方便吗？”,\n,\n\n---\n\n评估方法：,\n仔细阅读整段对话；,\n针对每一条评估标准逐项判断；,\n每项给出 1 或 0 分数，代表

## Model setup

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj",
                      "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True,
    loftq_config = None,
)

==((====))==  Unsloth 2025.5.8: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/168k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

Unsloth 2025.5.8 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


In [ ]:
from unsloth.chat_templates import standardize_sharegpt
from datasets import Dataset

train_set = standardize_sharegpt(Dataset.from_list(train_set))
prep_conversations = tokenizer.apply_chat_template(
    train_set["conversations"],
    add_generation_prompt=True,
    enable_thinking=False,
    tokenize = False,
)

Unsloth: Standardizing formats (num_proc=8):   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
data = pd.Series(prep_conversations)
data.name = "text"

combined_dataset = Dataset.from_pandas(pd.DataFrame(data))
combined_dataset = combined_dataset.shuffle(seed=3407)
combined_dataset[0]['text']

"<|im_start|>user\n### Instruction\n你是一名擅长评估银行客户经理（Relationship Manager，简称 RM）与客户之间对话的人工智能助手。\n\n你的任务：,\n阅读 RM 与客户的完整对话内容，并根据以下六项标准，判断 RM 是否按照规范完成自我介绍。每项符合标准记为 1 分，未符合标准记为 0 分。\n\n---\n\n评估标准（共 6 项）：,\n问候客户\n必须以礼貌方式向客户打招呼。,\n示例：”您好“、”早上好“、”哈喽，您好“。,\n,\n\n介绍自己的姓名（包括姓和名）\n只需要评估 RM 是否完整介绍自己，不需要考虑客户的姓名。,\n必须同时提供真实的名字和姓氏，昵称或仅名字不视为通过。,\n如果名字与姓氏连在一起且没有空格（如“张三丰”），则必须明确标注为姓与名的组合，否则不通过。,\n合格示例：“我叫张 伟”，“我是李 小美”。,\n不合格示例：“我叫小张”，“你可以叫我小王”，“我叫阿明”。,\n,\n\n说明所持许可证类型及有效的许可证编号\n必须明确提供许可证编号，并声明该许可证仍在有效期内。,\n示例：“我持有编号为 12345 的金融执业许可证，目前仍在有效期内。”,\n,\n\n说明此次会面的目的\n必须清楚说明此次会面目的，例如：投资组合更新、市场分析、新产品推荐等。,\n不允许使用模糊或空泛的表述，如“想来聊聊”、“简单交流一下”。,\n合格示例：“今天来是为了更新您的投资组合”、“我们来讨论一下当前市场走势”、“我想介绍一款新产品给您”。,\n不合格示例：“今天是来按预约见面”、“我有些事想跟您聊聊”或完全未提及会面目的。,\n,\n\n说明客户从此次会面中可获得的好处\n必须具体说明客户能从本次会面中获得什么收益。,\n示例：“这将有助于您优化投资结构”、“可以帮助提高投资回报率”。,\n,\n\n说明此次会面预计所需时间\n必须明确表示所需时间（可以是数字或文字），例如：15分钟、半小时、5分钟等。,\n若仅说“很快”、“不久”等模糊表述，则不算通过。,\n合格示例：“大约需要30分钟”、“预计聊1小时，您方便吗？”,\n,\n\n---\n\n评估方法：,\n仔细阅读整段对话；,\n针对每一条评估标准逐项判断；,\n每项给出 1 或 0 分数，代表是否符合要求。,\n\n#

## Model Training

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = combined_dataset,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 2,
        # max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
    ),
)

trainer_stats = trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/100 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100 | Num Epochs = 2 | Total steps = 24
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 128,450,560/14,000,000,000 (0.92% trained)


Step,Training Loss
1,1.760400
2,1.770000
3,1.453600
4,1.144600
5,0.860600
6,0.517500
7,0.297600
8,0.241200
9,0.275600
10,0.253700


In [ ]:
from google.colab import userdata

if False:
    model.save_pretrained("Qwen3-14B_transcribe", tokenizer, save_method="lora",)
if False:
    model.push_to_hub("PaweekornSora/Qwen3-14B_transcribe", save_method="lora",
                      token=userdata.get("HF_ADMIN"),
                      commit_message="Overwriting with latest version")
    tokenizer.push_to_hub("PaweekornSora/Qwen3-14B_transcribe", save_method="lora",
                      token=userdata.get("HF_ADMIN"),
                      commit_message="Overwriting with latest version")

README.md:   0%|          | 0.00/594 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/257M [00:00<?, ?B/s]

Saved model to https://huggingface.co/PaweekornSora/Qwen3-14B_transcribe


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

## Test set

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/SAI5_Onsite/Hack04_SCBx/test_transcribe.csv')
print(test_df.shape)
test_df.head()

(300, 10)


,id,first_name,last_name,กล่าวสวัสดี,แนะนำชื่อและนามสกุล,บอกประเภทใบอนุญาตและเลขที่ใบอนุญาตที่ยังไม่หมดอายุ,บอกวัตถุประสงค์ของการเข้าพบครั้งนี้,เน้นประโยชน์ว่าลูกค้าได้ประโยชน์อะไรจากการเข้าพบครั้งนี้,บอกระยะเวลาที่ใช้ในการเข้าพบ,transcribe
0,2e378ce7-b5f5-49b7-a337-0bc64b1115cd,วิมลศิริ,บุณยะลักษณ,False,False,False,True,True,False,มินท์มาคุยกับคุณวรรณนิดาในวันนี้เพื่ออยากจะแนะ...
1,70e43e4b-42f2-40f8-9822-2ec5ee0c94d4,จิราลักษ์,วารินทุ,False,False,True,False,False,True,ดิฉันชื่อจิราเป็นที่ปรึกษาทางการเงินค่ะ ก่อนเร...
2,b1b69e87-0a65-4aac-9eeb-910328f8bb1f,ปุณฐ์พัฒน์,อนัคฆฑัต,False,False,True,False,False,False,ผมปั๊บครับ เป็นที่ประสาททางการเงินแก่ธนาคารไทย...
3,292e0419-aed1-43d0-aa60-9635dc80574b,วฤณดา,มหาขันธ์,NaN,NaN,NaN,NaN,NaN,NaN,สวัสดีค่ะ คุณบุญธิชา ดิฉันชื่อวินดาหรือเรียกว่...
4,51f656f7-7558-4e4b-ab0a-fa91a772d196,ภัทรดา,อินทะวิวัฒนะ,NaN,NaN,NaN,NaN,NaN,NaN,ดิฉันชื่อ พัตรา อินตวิวัฒนธ์ เป็นที่ปรึกษาทางก...


In [ ]:
test_dataset = []
for i, row in test_df.iterrows():
    message = {
        'conversations': [
            {"role" : "user", "content" : prompt.format(row['transcribe'])},
        ]}
    test_dataset.append(message)


test_dataset = Dataset.from_list(test_dataset)
test_dataset = standardize_sharegpt(test_dataset)
test_dataset = tokenizer.apply_chat_template(
    test_dataset["conversations"],
    add_generation_prompt=True,
    enable_thinking=False,
    tokenize = False,
)
test_dataset[0]

Unsloth: Standardizing formats (num_proc=8):   0%|          | 0/300 [00:00<?, ? examples/s]

'<|im_start|>user\n### Instruction\n你是一名擅长评估银行客户经理（Relationship Manager，简称 RM）与客户之间对话的人工智能助手。\n\n你的任务：,\n阅读 RM 与客户的完整对话内容，并根据以下六项标准，判断 RM 是否按照规范完成自我介绍。每项符合标准记为 1 分，未符合标准记为 0 分。\n\n---\n\n评估标准（共 6 项）：,\n问候客户\n必须以礼貌方式向客户打招呼。,\n示例：”您好“、”早上好“、”哈喽，您好“。,\n,\n\n介绍自己的姓名（包括姓和名）\n只需要评估 RM 是否完整介绍自己，不需要考虑客户的姓名。,\n必须同时提供真实的名字和姓氏，昵称或仅名字不视为通过。,\n如果名字与姓氏连在一起且没有空格（如“张三丰”），则必须明确标注为姓与名的组合，否则不通过。,\n合格示例：“我叫张 伟”，“我是李 小美”。,\n不合格示例：“我叫小张”，“你可以叫我小王”，“我叫阿明”。,\n,\n\n说明所持许可证类型及有效的许可证编号\n必须明确提供许可证编号，并声明该许可证仍在有效期内。,\n示例：“我持有编号为 12345 的金融执业许可证，目前仍在有效期内。”,\n,\n\n说明此次会面的目的\n必须清楚说明此次会面目的，例如：投资组合更新、市场分析、新产品推荐等。,\n不允许使用模糊或空泛的表述，如“想来聊聊”、“简单交流一下”。,\n合格示例：“今天来是为了更新您的投资组合”、“我们来讨论一下当前市场走势”、“我想介绍一款新产品给您”。,\n不合格示例：“今天是来按预约见面”、“我有些事想跟您聊聊”或完全未提及会面目的。,\n,\n\n说明客户从此次会面中可获得的好处\n必须具体说明客户能从本次会面中获得什么收益。,\n示例：“这将有助于您优化投资结构”、“可以帮助提高投资回报率”。,\n,\n\n说明此次会面预计所需时间\n必须明确表示所需时间（可以是数字或文字），例如：15分钟、半小时、5分钟等。,\n若仅说“很快”、“不久”等模糊表述，则不算通过。,\n合格示例：“大约需要30分钟”、“预计聊1小时，您方便吗？”,\n,\n\n---\n\n评估方法：,\n仔细阅读整段对话；,\n针对每一条评估标准逐项判断；,\n每项给出 1 或 0 分数，代表是否符合要求。,\n\n#

In [ ]:
from transformers import TextStreamer

result = []
for text in tqdm(test_dataset):
    answer = model.generate(
        **tokenizer(text, return_tensors = "pt").to("cuda"),
        max_new_tokens = 2048, # Increase for longer outputs!
        temperature = 0.2, top_p = 0.8, top_k = 20, # For non thinking
        streamer = TextStreamer(tokenizer, skip_prompt = True),
    )

    decoded = tokenizer.decode(answer[0], skip_special_tokens=True)
    result.append(decoded)

  0%|          | 0/300 [00:00<?, ?it/s]

<|im_start|>assistant
<think>

</think>

{'is_greeting': False, 'is_introself': False, 'is_informlicense': False, 'is_informobjective': True, 'is_informbenefit': True, 'is_informinterval': False}<|im_end|>


  0%|          | 1/300 [00:08<40:27,  8.12s/it]

<|im_start|>assistant
<think>

</think>

{'is_greeting': False, 'is_introself': False, 'is_informlicense': True, 'is_informobjective': False, 'is_informbenefit': False, 'is_informinterval': True}<|im_end|>


  1%|          | 2/300 [00:15<38:57,  7.84s/it]

...

 99%|█████████▉| 298/300 [40:27<00:16,  8.18s/it]

<|im_start|>assistant
<think>

</think>

{'is_greeting': True, 'is_introself': False, 'is_informlicense': False, 'is_informobjective': True, 'is_informbenefit': False, 'is_informinterval': True}<|im_end|>


100%|█████████▉| 299/300 [40:35<00:08,  8.21s/it]

<|im_start|>assistant
<think>

</think>

{'is_greeting': False, 'is_introself': False, 'is_informlicense': True, 'is_informobjective': True, 'is_informbenefit': True, 'is_informinterval': False}<|im_end|>


100%|██████████| 300/300 [40:44<00:00,  8.15s/it]


In [ ]:
import ast

answer = None
for r in result:
    answer_dict = ast.literal_eval(r.split('</think>\n\n')[-1])
    label = list(answer_dict.values())
    answer = np.array(label) if answer is None else np.vstack([answer, label])

answer.shape

(300, 6)

In [ ]:
test_df.iloc[3:, 3:9] = answer[3:, :]
test_df.head()

,id,first_name,last_name,กล่าวสวัสดี,แนะนำชื่อและนามสกุล,บอกประเภทใบอนุญาตและเลขที่ใบอนุญาตที่ยังไม่หมดอายุ,บอกวัตถุประสงค์ของการเข้าพบครั้งนี้,เน้นประโยชน์ว่าลูกค้าได้ประโยชน์อะไรจากการเข้าพบครั้งนี้,บอกระยะเวลาที่ใช้ในการเข้าพบ,transcribe
0,2e378ce7-b5f5-49b7-a337-0bc64b1115cd,วิมลศิริ,บุณยะลักษณ,False,False,False,True,True,False,มินท์มาคุยกับคุณวรรณนิดาในวันนี้เพื่ออยากจะแนะ...
1,70e43e4b-42f2-40f8-9822-2ec5ee0c94d4,จิราลักษ์,วารินทุ,False,False,True,False,False,True,ดิฉันชื่อจิราเป็นที่ปรึกษาทางการเงินค่ะ ก่อนเร...
2,b1b69e87-0a65-4aac-9eeb-910328f8bb1f,ปุณฐ์พัฒน์,อนัคฆฑัต,False,False,True,False,False,False,ผมปั๊บครับ เป็นที่ประสาททางการเงินแก่ธนาคารไทย...
3,292e0419-aed1-43d0-aa60-9635dc80574b,วฤณดา,มหาขันธ์,True,False,True,True,True,False,สวัสดีค่ะ คุณบุญธิชา ดิฉันชื่อวินดาหรือเรียกว่...
4,51f656f7-7558-4e4b-ab0a-fa91a772d196,ภัทรดา,อินทะวิวัฒนะ,False,False,True,True,False,False,ดิฉันชื่อ พัตรา อินตวิวัฒนธ์ เป็นที่ปรึกษาทางก...


In [ ]:
test_df.drop(columns='transcribe').to_csv('ASR_Qwen14B_fine-tuned.csv', index=False)